In [6]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import time

In [14]:
def get_historical_data(unit, count, to=None):
    url = f"https://api.upbit.com/v1/candles/minutes/{unit}?market=KRW-BTC&to={to}&count={count}"
    headers = {"accept": "application/json"}

    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Error: {response.json()}")
    data = response.json()
    return data

def fetch_all_historical_data(unit, start_date, end_date):
    start = datetime.strptime(start_date, '%Y-%m-%dT %H:%M:%S')
    all_data = []
    to_date = start.strftime('%Y-%m-%dT %H:%M:%S')

    end = datetime.strptime(end_date, '%Y-%m-%dT %H:%M:%S')

    while datetime.strptime(to_date, '%Y-%m-%dT %H:%M:%S') > end:
        data = get_historical_data(unit=unit, count=200, to=to_date)
        print(data)
        break
        all_data.extend(data)
        last_candle_time = datetime.strptime(data[-1]['candle_date_time_utc'], '%Y-%m-%dT %H:%M:%S')
        to_date = (last_candle_time - timedelta(minutes=unit)).strftime('%Y-%m-%dT %H:%M:%S')
        time.sleep(1)
        
    return all_data

# Example: Fetch all 1-minute interval data from 2024-06-15 to 2017-10-01
end_date = '2017-10-01T 00:00:00'
start_date = '2024-06-15T 23:59:59'
historical_data = fetch_all_historical_data(unit=60, start_date=start_date, end_date=end_date)


Error: {'error': {'name': 400, 'message': 'Invalid parameter. Check the given value!'}}
{'error': {'name': 400, 'message': 'Invalid parameter. Check the given value!'}}


In [21]:
df=pd.DataFrame(historical_data)
df['candle_date_time_kst'] = pd.to_datetime(df['candle_date_time_kst'], format='%Y-%m-%dT%H:%M:%S')
df = df.sort_values(by='candle_date_time_kst')
# df = df.drop_duplicates().reset_index(drop=True)
df.reset_index(drop=True)

,market,candle_date_time_utc,candle_date_time_kst,opening_price,high_price,low_price,trade_price,timestamp,candle_acc_trade_price,candle_acc_trade_volume,unit
0,KRW-BTC,2024-06-08T08:00:00,2024-06-08 17:00:00,98004000.0,98020000.0,97881000.0,97883000.0,1717837199888,3.552669e+09,36.266770,60
1,KRW-BTC,2024-06-08T09:00:00,2024-06-08 18:00:00,97883000.0,98007000.0,97869000.0,97994000.0,1717840798487,3.181803e+09,32.494931,60
2,KRW-BTC,2024-06-08T10:00:00,2024-06-08 19:00:00,97994000.0,98000000.0,97864000.0,97978000.0,1717844398566,3.511806e+09,35.859602,60
3,KRW-BTC,2024-06-08T11:00:00,2024-06-08 20:00:00,97979000.0,98114000.0,97848000.0,97870000.0,1717847999908,8.200075e+09,83.681919,60
4,KRW-BTC,2024-06-08T12:00:00,2024-06-08 21:00:00,97870000.0,98030000.0,97800000.0,98020000.0,1717851599597,6.765931e+09,69.100840,60
...,...,...,...,...,...,...,...,...,...,...,...
195,KRW-BTC,2024-06-16T11:00:00,2024-06-16 20:00:00,94370000.0,94490000.0,94176000.0,94180000.0,1718539198131,7.001891e+09,74.212920,60
196,KRW-BTC,2024-06-16T12:00:00,2024-06-16 21:00:00,94180000.0,94434000.0,94178000.0,94300000.0,1718542799647,3.778196e+09,40.069316,60
197,KRW-BTC,2024-06-16T13:00:00,2024-06-16 22:00:00,94300000.0,94597000.0,94218000.0,94593000.0,1718546397181,6.225354e+09,65.946532,60
198,KRW-BTC,2024-06-16T14:00:00,2024-06-16 23:00:00,94592000.0,94593000.0,94320000.0,94350000.0,1718549998509,4.547740e+09,48.149277,60
